# The Lorenz Attractor

#### Patrick Harrison 3002363<hr>


### Introduction

The Lorenz attractor (butterfly) is one of the more famous chaotic systems in physics and can be calculated with the system of differential equations $\mathcal{L}$ as

\begin{align}
    \mathcal{L}=
    \begin{cases}
        \dot{X} &= \sigma(Y-X)\\
        \dot{Y} &= -XZ+\rho X-Y\\
        \dot{Z} &= XY-\beta Z
    \end{cases}
\end{align}

here $\sigma$, $\rho$ and $\beta$ are physical parameters. This system of equations is non-linear, deterministic and for certain parameters, the system is also non-periodic and chaotic. Lorenz used parameters:
\begin{align}
    \sigma=10 && \rho=28 && \beta=8/3
\end{align}
It is found these and nearby parameters will reamin chaotic and non-periodic. [*]

The problem here is that extracting the physical parameters given some data may not always be easy or obvious. If you knew the phsyical parameters the deterministic syustem could be solved and predicted. If say you are given a time series of states, can you predict the next states without knowing the physical parameters. This becomes a superivsed machine lenaring problem.

This notebook walks through the Lorenz attractor and explores useful properties that may help when creating a nerual network.

<hr> 
[*] Hirsch, Morris W.; Smale, Stephen; Devaney, Robert (2003). Differential Equations, Dynamical Systems, & An Introduction to Chaos (Second ed.). Boston, MA: Academic Press. ISBN 978-0-12-349703-1.

### Exploring The Lorenz Attractor

Using numpy, scipy and matplotlib, The Lorenz attractor is easially calcualted for given physical parameters. See the Lorenz module for calculation. The states are numerically solved using `scipy.integrate.odeint` wich impliments a numerical jacobian solver for the system. 

In [1]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.gridspec import GridSpec
import matplotlib.animation as animation

# Use tk if sliders are not working in notebook
%matplotlib widget

import numpy as np
from Lorenz import LorenzAttractor

plt.style.use("astro") # comment out if you dont have my astro mpl style sheet

In [2]:
# Set up Lorenz attractor shape and initial parameters
INITIALSTATE = (1.0, 1.0, 1.0)
SIGMA = 10.0
BETA = 8/3.0
RHO = 28.0

# Set integration time sequence for solveing
tStart = 0.0
tEnd = 30.0
dt = 0.003

t = np.arange(tStart, tEnd, dt)

# Numerically solve
LA = LorenzAttractor(t, INITIALSTATE, SIGMA, BETA, RHO)
X, Y, Z = LA.get_states()

In [3]:
print(LA)

 Lorenz Attractor: (s=10.0, b=2.6666666666666665, 
                    r=28.0), L0=(1.0, 1.0, 1.0)
                


The matplotlib sliders lets us explore how different physical parameters change the features in the shape. The critical points (definined for $\rho>1$) are plotted in red as well.

In [4]:
fig, ax, sliders = LA.plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Plotting the Lorenz attractor in physical space. Here we use an animation to make the chaotic nature of the trajectory clear. Again if the animation is not working try the `%matplotlib tk` magic command to plot in a seperate window.

Using a scipy library, the physical trajector is solved from the system of ordinary differential equations listed above. For the rest of this notebook, unless stated, our initial state will be $[x,y,z]=[1.0, 1.0, 1.0]$ with the following physical parameters.

\begin{align*}
    \sigma=10.0 && \beta=\frac{8}{3} && \rho=28.0
\end{align*}
These numbers (and numbers around these points) produce a chaotic attractor (strange attractor) [1].

In [5]:
''' Animation of the Lorenz attractor '''

#%matplotlib tk # animation in a tkinter window

fig = plt.figure()
ax = fig.gca(projection='3d')

def initAnimation():
    trajectory.set_data(X[0],Y[0])
    trajectory.set_3d_properties(Z[0])
    return trajectory,

def animate(i):
    trajectory.set_data(X[:i], Y[:i])
    trajectory.set_3d_properties(Z[:i])
    return trajectory,

trajectory, = ax.plot(X[0], Y[0], Z[0], lw=0.75)
# call the animator.
anim = animation.FuncAnimation(fig, animate, init_func=initAnimation,
                               frames=len(X), interval=10, blit=True, repeat=False)


title = r"Lorenz attractor: $\sigma=${}, $\beta=${}, $\rho=${}"
ax.set_xlabel("X Axis")
ax.set_ylabel("Y Axis")
ax.set_zlabel("Z Axis")
ax.set_title(title.format(str(LA._sigma),str(LA._beta),str(LA._rho)))

# For some reason matplotlib 3d projection is not 
# autochanging the bounds
ax.set_xlim(-20, 20)
ax.set_ylim(-30, 30)
ax.set_zlim(0, 50)

#ax.set_axis_off()
plt.show()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Small perterbations in physical parameters and initial conditions can drastically change the shape of the trajectory.

### Autocorrelation for maximum time lag

In [6]:
def acf(x, length=20):
    return np.array([1]+[np.corrcoef(x[:-i], x[i:])[0,1]  \
        for i in range(1, length)])

In [7]:
fig, ax = plt.subplots(1, 1, figsize=plt.figaspect(.75))

acX = acf(X, length=200)
acY = acf(Y, length=200)
acZ = acf(Z, length=200)

lagX = np.where(acX == acX.min())[0][0]
lagY = np.where(acY == acY.min())[0][0]
lagZ = np.where(acZ == acZ.min())[0][0]

ax.annotate(str(lagX), (lagX, acX.min()))
ax.annotate(str(lagY), (lagY, acY.min()))
ax.annotate(str(lagZ), (lagZ, acZ.min()))

ax.plot(acX, label="X")
ax.plot(acY, label="Y")
ax.plot(acZ, label="Z")

ax.set_title("Autocorrelation for each dimenstion")
ax.set_xlabel("timelag")
ax.set_ylabel("Correlation Coefficient")
plt.legend()

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:
print("X first minimum correlation at:", lagX)
print("Y minimum correlation at:", lagY)
print("Z minimum correlation at:", lagZ)

X first minimum correlation at: 163
Y minimum correlation at: 138
Z minimum correlation at: 120


This shows how much effect the last states have on the current state. A time lag, when predicting a model, larger than the minimum coefficient, would not be effective as the correlation has passed a minimum.

Mutual information between the dimensions would also be interesting to look at and may help when deciding how to structure the Neural Network.

There is a couple of ideas that can be explored here. For example a time series prediction of the next steps of an attractor or a parameter estimation given the trajectory. Here a perceptron nerual network and a Recusive Neural Network is presented to give a prediction on the next states.